In [1]:
import numpy as np
import pandas as pd
import datetime    
import torch
import torch.nn as nn
import torch.utils.data.dataloader
from neural_net import P1_Net, do_train

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.utils.validation import column_or_1d

In [2]:
df_city = pd.read_csv("city_attributes.csv")
df_humidity = pd.read_csv("humidity.csv")
df_pressure = pd.read_csv("pressure.csv")
df_temper = pd.read_csv("temperature.csv")
df_descript = pd.read_csv("weather_description.csv")
df_direction = pd.read_csv("wind_direction.csv")
df_speed = pd.read_csv("wind_speed.csv")

In [3]:
max([df_humidity.isna().sum().mean(),
df_pressure.isna().sum().mean(),
df_temper.isna().sum().mean(),
df_direction.isna().sum().mean(),
df_speed.isna().sum().mean()])

774.3513513513514

In [4]:
df_temper.describe(include='all')

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
count,45253,44458.000000,45252.000000,44460.000000,45250.000000,45250.000000,45252.000000,45252.000000,45250.000000,45252.000000,...,45250.000000,44460.000000,45250.000000,45250.000000,44455.000000,44460.000000,44461.000000,44455.000000,44456.000000,44460.000000
unique,45253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2012-10-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,283.862654,284.992929,288.155821,284.409626,290.846116,290.215044,292.424887,295.493358,285.617856,...,285.374168,285.400406,280.343010,283.779823,291.521986,294.512307,296.497276,295.266398,294.094803,293.184253
std,NaN,6.640131,7.452438,5.332862,6.547986,6.460823,5.889992,10.829522,9.916743,9.853484,...,10.242377,10.220932,11.953626,9.802499,7.821815,6.676412,8.852984,6.324566,6.304118,7.093583
min,NaN,245.150000,262.370000,272.300000,263.780000,266.503667,265.783333,260.561333,266.059000,255.042333,...,250.390000,250.774000,243.300000,249.540000,272.179000,271.049000,271.150000,271.150000,268.682000,272.974000
25%,NaN,279.160000,279.850000,284.670000,279.830000,286.380000,286.254750,283.920000,287.680000,277.970000,...,277.350636,277.370000,271.971750,276.090000,285.366623,289.450000,289.734000,290.467333,289.881833,287.524279
50%,NaN,283.450000,284.320000,287.610000,283.940000,290.530000,290.118750,292.027486,295.586667,286.120000,...,285.927583,285.870000,281.109000,284.133250,290.932667,294.900000,296.150000,294.820000,294.150000,292.996000
75%,NaN,288.600785,289.451750,291.015167,288.530000,295.080000,294.107542,300.835000,303.050000,292.835643,...,293.796000,293.760000,290.369583,291.620000,297.270000,299.800000,303.150000,299.660000,298.930000,299.150000


In [5]:
def group_by_days(df):
    df["date"] = pd.to_datetime(df["datetime"]).dt.date
    df = df.drop('datetime', axis=1)
    df = df.groupby(df["date"]).mean()
    return df

def group_by_days_descript(df):
    df["date"] = pd.to_datetime(df["datetime"]).dt.date
    df = df.fillna('no data')
    df = df.drop('datetime', axis=1)
    df = df.groupby(df["date"]).agg(lambda x: pd.Series.mode(x)[0])
    return df

In [6]:
df_humidity = group_by_days(df_humidity)
df_pressure = group_by_days(df_pressure)
df_temper = group_by_days(df_temper)
df_descript = group_by_days_descript(df_descript)
df_direction = group_by_days(df_direction)
df_speed = group_by_days(df_speed)

In [7]:
encoder = LabelEncoder()
df_descript= df_descript.apply(encoder.fit_transform)

In [8]:
def create_dataframe_for_city(city_name):
    df =  pd.concat([
        df_humidity[city_name], df_pressure[city_name], df_temper[city_name], df_descript[city_name], df_direction[city_name], df_speed[city_name]
    ], axis=1)
    df.columns = ['humidity', 'pressure', 'temperature', 'description', 'wind_direction','wind_speed' ]
    return df

In [9]:
df_Portland  =  create_dataframe_for_city("Portland")
df_Portland

,humidity,pressure,temperature,description,wind_direction,wind_speed
date,,,,,,
2012-10-01,78.727273,1024.000000,282.118197,11,57.727273,0.000000
2012-10-02,65.833333,1023.583333,286.137728,11,214.041667,1.291667
2012-10-03,66.208333,1021.083333,289.599792,2,228.333333,2.625000
2012-10-04,51.166667,1022.875000,286.482500,12,206.750000,4.625000
2012-10-05,40.391304,1022.916667,288.286042,12,182.250000,3.708333
...,...,...,...,...,...,...
2017-11-26,87.625000,1004.500000,281.588333,8,153.750000,4.958333
2017-11-27,92.875000,1015.625000,279.922500,8,158.083333,1.666667
2017-11-28,88.458333,1019.791667,280.085833,8,144.166667,3.041667


In [10]:
df_speed_2 = pd.read_csv("wind_speed.csv")
df_speed_2["date"] = pd.to_datetime(df_speed_2["datetime"]).dt.date
df_speed_2 = df_speed_2.drop(columns=['datetime'])
df_speed_2 = df_speed_2.groupby('date').apply(lambda group: (group >= 6).any())
strong_wind = pd.concat([
    df_speed_2[city] for city in df_city['City']
])

def add_city_col(df, city):
    df['city'] = city
    return df
all_cities_df = pd.concat([
    add_city_col(create_dataframe_for_city(city), city) for city in df_city['City']
])
all_cities_df['strong_wind'] = strong_wind
all_cities_df['strong_wind'] = all_cities_df['strong_wind'].apply(lambda x: 1 if x else 0)
all_cities_df = all_cities_df.dropna().sort_values('date', kind='stable')

In [11]:
all_cities_df

,humidity,pressure,temperature,description,wind_direction,wind_speed,city,strong_wind
date,,,,,,,,
2012-10-01,78.727273,1024.000000,282.118197,11,57.727273,0.000000,Portland,0
2012-10-01,83.000000,1009.727273,289.416642,14,122.363636,1.636364,San Francisco,0
2012-10-01,78.000000,1030.000000,281.767262,1,32.272727,0.000000,Seattle,0
2012-10-01,88.000000,1013.000000,291.846501,11,0.000000,0.000000,Los Angeles,0
2012-10-01,79.909091,1013.000000,291.573495,11,0.000000,0.000000,San Diego,0
...,...,...,...,...,...,...,...,...
2017-11-30,42.000000,1025.000000,279.190000,12,360.000000,2.000000,Pittsburgh,0
2017-11-30,60.000000,1027.000000,274.510000,17,330.000000,3.000000,Toronto,0
2017-11-30,32.000000,1024.000000,283.420000,11,360.000000,4.000000,Philadelphia,0


# Przewidywanie temperatury

In [12]:
def extract_x_y_temperature(df):
    X = df.iloc[:-2].copy()
    y =  df['temperature'].iloc[4:] 

    X.reset_index(inplace= True)
    X = X.drop(columns = ['strong_wind'])
    X['day_of_year'] = pd.to_datetime(X['date']).dt.dayofyear
    X['month'] = pd.to_datetime(X['date']).dt.month

    # X['wind_dir_sin'] = np.sin(X['wind_direction']/360 * 2 * np.pi)
    # X['wind_dir_cos'] = np.cos(X['wind_direction']/360 * 2 * np.pi)
    # X['day_sin'] = np.sin(X['day_of_year']/365 * 2 * np.pi)
    # X['day_cos'] = np.cos(X['day_of_year']/365 * 2 * np.pi)
    # X = X.drop(columns = ['day_of_year'])
    # X = X.drop(columns = ['wind_direction'])

    X = X.drop('date', axis=1)
    column_names = X.columns.to_list()
    X = pd.concat([X.iloc[:-2].reset_index(drop=True),X.iloc[1:-1].reset_index(drop=True),X.iloc[2:].reset_index(drop=True)],axis = 1)
    X.columns = column_names + [c + '_1' for c in column_names] + [c + '_2' for c in column_names]
    if 'city_1' in X.columns:
        X = X.drop(columns=['city_1', 'city_2'])
    return X, y

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def gen_col_set(col_names):
    return [ final_col
        for col_name in col_names
        for final_col in  [col_name, col_name+"_1", col_name+"_2"] ]

def get_col_transformer_temp(df):
    return ColumnTransformer(
        [
            (
                "standarizer",
                StandardScaler(),
                gen_col_set(
                    ["humidity", "pressure", "temperature", "wind_speed", "wind_direction", "day_of_year"]
                ),
            ),
            (
                "one_hot_encoder", 
                OneHotEncoder(), 
                gen_col_set(
                    ["description", "month"]
                ) + (['city'] if 'city' in df.columns else [])
            ),
        ],
        remainder="passthrough",
    )

def prepare_temperature_data(df):
    X, y = extract_x_y_temperature(df)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size = 0.3, random_state = 0,shuffle=False)
    ct = get_col_transformer_temp(X_train)
    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)
    return X_train, X_test, y_train, y_test

In [14]:
X_train, X_test, y_train, y_test = prepare_temperature_data(all_cities_df)
X_train.shape

(47273, 150)

In [15]:

dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.toarray().astype('float32')), 
    torch.from_numpy(y_train.to_numpy().astype('float32')).unsqueeze(1)
)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
net = P1_Net(
    nn.Sequential(
        nn.Linear(X_train.shape[1], 256),
        nn.ReLU(),
        nn.Linear(256,1)
    )
)
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
loss = nn.MSELoss()

do_train(net, data_loader, optimizer, loss, 400)

P1_Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=150, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=1, bias=True)
  )
)
[1] loss: 82686.843
[2] loss: 82405.617
[3] loss: 82022.465
[4] loss: 81475.271
[5] loss: 80716.951
[6] loss: 79718.424
[7] loss: 78466.651
[8] loss: 76959.550
[9] loss: 75203.530
[10] loss: 73212.039
[11] loss: 71000.358
[12] loss: 68583.536
[13] loss: 65983.515
[14] loss: 63228.901
[15] loss: 60351.036
[16] loss: 57380.240
[17] loss: 54343.127
[18] loss: 51263.380
[19] loss: 48163.788
[20] loss: 45066.537
[21] loss: 41992.596
[22] loss: 38960.213
[23] loss: 35981.907
[24] loss: 33065.781
[25] loss: 30223.519
[26] loss: 27471.637
[27] loss: 24827.166
[28] loss: 22303.353
[29] loss: 19910.804
[30] loss: 17657.044
[31] loss: 15548.270
[32] loss: 13590.345
[33] loss: 11786.520
[34] loss: 10138.075
[35] loss: 8644.302
[36] loss: 7302.280
[37] loss: 6108.438
[38] loss: 5057.386
[39] loss: 4141.433
[40] los

In [16]:
from sklearn.metrics import mean_absolute_error
net.eval()

def valid_in_margin(y1, y2, margin):
    return 1 - np.count_nonzero(abs(y1-y2)>margin) / len(y1)
print("* train")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_train.toarray().astype('float32'))).numpy()
    y_pred1 = np.squeeze(y_pred1)

mae = mean_absolute_error(y_train, y_pred1)
print(f"\t* Mean absolute error: {mae}")
margin = 2
fraction = valid_in_margin(y_pred1, y_train, margin)
print(f"\t* Fraction of predictions with absolute error <= {margin}: {fraction}")

print("* test")
with torch.no_grad():
    y_pred2 = net(torch.from_numpy(X_test.toarray().astype('float32'))).numpy()
    y_pred2 = np.squeeze(y_pred2)

mae = mean_absolute_error(y_test, y_pred2)
print(f"\t* Mean absolute error: {mae}")
margin = 2
fraction = valid_in_margin(y_pred2, y_test, margin)
print(f"\t* Fraction of predictions with absolute error <= {margin}: {fraction}")

* train
	* Mean absolute error: 2.224468288295613
	* Fraction of predictions with absolute error <= 2: 0.5564064053476614
* test
	* Mean absolute error: 2.190860158016228
	* Fraction of predictions with absolute error <= 2: 0.5778095849168353


In [26]:
X_train, X_test, y_train, y_test = prepare_temperature_data(all_cities_df)
X_train.shape



dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.toarray().astype('float32')), 
    torch.from_numpy(y_train.to_numpy().astype('float32')).unsqueeze(1)
)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
net = P1_Net(
    nn.Sequential(
        nn.Linear(X_train.shape[1], 256),
        nn.ReLU(),
        nn.Linear(256,1)
    )
)
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
loss = nn.L1Loss()

do_train(net, data_loader, optimizer, loss, 400)

P1_Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=150, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=1, bias=True)
  )
)
[1] loss: 287.397
[2] loss: 286.918
[3] loss: 286.291
[4] loss: 285.415
[5] loss: 284.211
[6] loss: 282.615
[7] loss: 280.581
[8] loss: 278.086
[9] loss: 275.126
[10] loss: 271.705
[11] loss: 267.825
[12] loss: 263.482
[13] loss: 258.674
[14] loss: 253.406
[15] loss: 247.688
[16] loss: 241.528
[17] loss: 234.927
[18] loss: 227.896
[19] loss: 220.438
[20] loss: 212.565
[21] loss: 204.281
[22] loss: 195.580
[23] loss: 186.467
[24] loss: 176.955
[25] loss: 167.061
[26] loss: 156.801
[27] loss: 146.192
[28] loss: 135.249
[29] loss: 123.980
[30] loss: 112.380
[31] loss: 100.440
[32] loss: 88.171
[33] loss: 75.587
[34] loss: 62.690
[35] loss: 49.487
[36] loss: 36.020
[37] loss: 23.027
[38] loss: 13.260
[39] loss: 8.650
[40] loss: 6.855
[41] loss: 5.804
[42] loss: 5.042
[43] loss: 4.489
[44] loss: 4.087
[45] 

In [28]:
from sklearn.metrics import mean_absolute_error

def valid_in_margin(y1, y2, margin):
    return 1 - np.count_nonzero(abs(y1-y2)>margin) / len(y1)
print("* train")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_train.toarray().astype('float32'))).numpy()
    y_pred1 = np.squeeze(y_pred1)

mae = mean_absolute_error(y_train, y_pred1)
print(f"\t* Mean absolute error: {mae}")
margin = 2
fraction = valid_in_margin(y_pred1, y_train, margin)
print(f"\t* Fraction of predictions with absolute error <= {margin}: {fraction}")

print("* test")
with torch.no_grad():
    y_pred2 = net(torch.from_numpy(X_test.toarray().astype('float32'))).numpy()
    y_pred2 = np.squeeze(y_pred2)

mae = mean_absolute_error(y_test, y_pred2)
print(f"\t* Mean absolute error: {mae}")
margin = 2
fraction = valid_in_margin(y_pred2, y_test, margin)
print(f"\t* Fraction of predictions with absolute error <= {margin}: {fraction}")

* train
	* Mean absolute error: 2.1749646674385086
	* Fraction of predictions with absolute error <= 2: 0.5805004971125167
* test
	* Mean absolute error: 2.1710920349148526
	* Fraction of predictions with absolute error <= 2: 0.5876314100982183


## Przewidywanie siły wiatru

In [17]:
def extract_x_y_wind(df):
    X = df.copy()
    X = X.iloc[:-2]
    y = df['strong_wind'].iloc[4:] 

    X.reset_index(inplace= True)
    X['day_of_year'] = pd.to_datetime(X['date']).dt.dayofyear
    X['month'] = pd.to_datetime(X['date']).dt.month
    # X['day_sin'] = np.sin(X['day_of_year']/365 * 2 * np.pi)
    # X['day_cos'] = np.cos(X['day_of_year']/365 * 2 * np.pi)
    # X = X.drop(columns = ['day_of_year'])


    X = X.drop('date', axis=1)
    column_names = X.columns.to_list()
    X= pd.concat([X.iloc[:-2].reset_index(drop=True),X.iloc[1:-1].reset_index(drop=True),X.iloc[2:].reset_index(drop=True)],axis = 1)
    X.columns = column_names + [c + '_1' for c in column_names] + [c + '_2' for c in column_names]
    if 'city_1' in X.columns:
        X = X.drop(columns=['city_1', 'city_2'])
    return X, y

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def get_col_transformer_wind(df):
    return ColumnTransformer(
        [
            (
                "standarizer",
                StandardScaler(),
                gen_col_set(
                    ["humidity", "pressure", "temperature", "wind_speed", "wind_direction", "day_of_year"]
                ),
            ),
            (
                "one_hot_encoder", 
                OneHotEncoder(), 
                gen_col_set(
                    ["description", "month"]
                ) + (['city'] if 'city' in df.columns else [])
            ),
            ('pass', 'passthrough', gen_col_set(['strong_wind'])),
        ],
        remainder="passthrough",
    )

def prepare_wind_data(df):
    X, y = extract_x_y_wind(df)
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size = 0.3, random_state = 0,shuffle=False)
    ct = get_col_transformer_wind(X_train)
    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)
    return X_train, X_test, y_train, y_test

In [19]:
X_train, X_test, y_train, y_test = prepare_wind_data(all_cities_df)
X_train.shape

(47273, 153)

In [ ]:
y_0 = np.array([1 if i==0 else 0 for i in y_train])
y_t = np.concatenate([np.expand_dims(y_0, axis=1),np.expand_dims(y_train, axis=1)],axis=1)
dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.toarray().astype('float32')), 
    torch.from_numpy(y_t.astype('float32'))
)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
net = P1_Net(
    nn.Sequential(
        nn.Linear(X_train.shape[1], 256),
        nn.ReLU(),
        nn.Linear(256,2),
        nn.Softmax(dim=1)
    )
)
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

do_train(net, data_loader, optimizer, loss, 200)

P1_Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=153, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)
[1] loss: 0.640
[2] loss: 0.605
[3] loss: 0.592
[4] loss: 0.584
[5] loss: 0.577
[6] loss: 0.572
[7] loss: 0.568
[8] loss: 0.564
[9] loss: 0.560
[10] loss: 0.556
[11] loss: 0.553
[12] loss: 0.550
[13] loss: 0.547
[14] loss: 0.544
[15] loss: 0.541
[16] loss: 0.538
[17] loss: 0.536
[18] loss: 0.533
[19] loss: 0.531
[20] loss: 0.529
[21] loss: 0.527
[22] loss: 0.525
[23] loss: 0.522
[24] loss: 0.522
[25] loss: 0.518
[26] loss: 0.521
[27] loss: 0.521
[28] loss: 0.519
[29] loss: 0.537
[30] loss: 0.519
[31] loss: 0.522
[32] loss: 0.515
[33] loss: 0.517
[34] loss: 0.512
[35] loss: 0.513
[36] loss: 0.508
[37] loss: 0.509
[38] loss: 0.504
[39] loss: 0.505
[40] loss: 0.515
[41] loss: 0.510
[42] loss: 0.517
[43] loss: 0.508
[44] loss: 0.504
[45] loss: 0.500
[46] loss: 0.499
[47] loss: 0.496

In [ ]:
from sklearn.metrics import roc_auc_score
net.eval()
print("* train")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_train.toarray().astype('float32')))[:,1]
print(f"\t* ROC_AUC: {roc_auc_score(y_train, y_pred1)}")

print("* test")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_test.toarray().astype('float32')))[:,1]
print(f"\t* ROC_AUC: {roc_auc_score(y_test, y_pred1)}")

In [38]:
class FocalLoss(nn.Module):
    """Implementation of Facal Loss"""
    def __init__(self, weight=None, gamma=2, reduction="mean"):
        super(FocalLoss, self).__init__()
        self.weighted_cs = nn.CrossEntropyLoss(weight=weight, reduction="none")
        self.cs = nn.CrossEntropyLoss(reduction="none")
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, predicted, target):
        """
        predicted: [batch_size, n_classes]
        target: [batch_size]
        """
        pt = 1/torch.exp(self.cs(predicted,target))
        #shape: [batch_size]
        entropy_loss = self.weighted_cs(predicted, target)
        #shape: [batch_size]
        focal_loss = ((1-pt)**self.gamma)*entropy_loss
        #shape: [batch_size]
        if self.reduction =="none":
            return focal_loss
        elif self.reduction == "mean":
            return focal_loss.mean()
        else:
            return focal_loss.sum()

In [39]:
X_train, X_test, y_train, y_test = prepare_wind_data(all_cities_df)
X_train.shape
y_0 = np.array([1 if i==0 else 0 for i in y_train])
y_t = np.concatenate([np.expand_dims(y_0, axis=1),np.expand_dims(y_train, axis=1)],axis=1)
dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.toarray().astype('float32')), 
    torch.from_numpy(y_t.astype('float32'))
)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
net = P1_Net(
    nn.Sequential(
        nn.Linear(X_train.shape[1], 256),
        nn.ReLU(),
        nn.Linear(256,2),
        nn.Softmax(dim=1)
    )
)
print(net)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss = FocalLoss()

do_train(net, data_loader, optimizer, loss, 200)

P1_Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=153, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)
[1] loss: 0.158
[2] loss: 0.149
[3] loss: 0.146
[4] loss: 0.143
[5] loss: 0.140
[6] loss: 0.138
[7] loss: 0.137
[8] loss: 0.135
[9] loss: 0.133
[10] loss: 0.132
[11] loss: 0.130
[12] loss: 0.129
[13] loss: 0.128
[14] loss: 0.126
[15] loss: 0.125
[16] loss: 0.124
[17] loss: 0.123
[18] loss: 0.121
[19] loss: 0.120
[20] loss: 0.120
[21] loss: 0.118
[22] loss: 0.117
[23] loss: 0.118
[24] loss: 0.125
[25] loss: 0.117
[26] loss: 0.120
[27] loss: 0.116
[28] loss: 0.116
[29] loss: 0.115
[30] loss: 0.115
[31] loss: 0.114
[32] loss: 0.115
[33] loss: 0.113
[34] loss: 0.114
[35] loss: 0.111
[36] loss: 0.111
[37] loss: 0.108
[38] loss: 0.108
[39] loss: 0.106
[40] loss: 0.109
[41] loss: 0.122
[42] loss: 0.110
[43] loss: 0.108
[44] loss: 0.105
[45] loss: 0.105
[46] loss: 0.104
[47] loss: 0.103

In [40]:
from sklearn.metrics import roc_auc_score
net.eval()
print("* train")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_train.toarray().astype('float32')))[:,1]
print(f"\t* ROC_AUC: {roc_auc_score(y_train, y_pred1)}")

print("* test")
with torch.no_grad():
    y_pred1 = net(torch.from_numpy(X_test.toarray().astype('float32')))[:,1]
print(f"\t* ROC_AUC: {roc_auc_score(y_test, y_pred1)}")

* train
	* ROC_AUC: 0.9678768633465775
* test
	* ROC_AUC: 0.7364716928598911
